In [130]:
!pip install -U sagemaker

In [131]:
import sagemaker
import requests

In [132]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel

In [133]:
import os
import boto3
import time

In [134]:
sess = boto3.Session()
sm = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [135]:
!ls -al

total 76
drwxr-xr-x  5 root root     6144 Dec 21 09:50 .
drwx------ 14 root nogroup  6144 Dec 21 14:18 ..
drwxr-xr-x  2 root root     6144 Dec 21 02:19 .ipynb_checkpoints
drwxr-xr-x  4 root root     6144 Nov 23 03:10 Extras
drwxr-xr-x  5 root root     6144 Dec 14 19:58 UWH-6
-rw-r--r--  1 root root     2793 Dec 21 09:50 inference.py
-rw-r--r--  1 root root     2801 Dec 21 09:50 inference_2.py
-rw-r--r--  1 root root     2185 Dec 21 09:50 inference_3.py
-rw-r--r--  1 root root     1961 Sep 27 01:13 input_fn.py
-rw-r--r--  1 root root    17695 Dec 21 09:50 yolov7-Copy1.ipynb
-rw-r--r--  1 root root    20024 Nov 23 03:10 yolov7-Copy2_preprocessing.ipynb


In [136]:
# Download YOLOv7 repository from Roboflow-ai and install requirements
!git clone https://github.com/roboflow-ai/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 630, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 630 (delta 1), reused 16 (delta 1), pack-reused 612
Receiving objects: 100% (630/630), 107.03 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (306/306), done.
Checking out files: 100% (118/118), done.


In [138]:
%cd yolov7

/root/archieves/yolov7


In [139]:
!mkdir sm-deploy

In [140]:
!mv runs/best.pt requirements.txt sm-deploy/

In [142]:
#Set a default S3 bucket
default_bucket = sagemaker_session.default_bucket()
default_bucket

'sagemaker-us-west-2-196327195270'

In [150]:
%cd ..

/root/archieves/yolov7


In [151]:
BUCKET = os.path.join("s3://", default_bucket)
DATA_PATH = os.path.join(BUCKET, "images")
IMAGE_NAME = "inference/images/swimmers/Angers19_100-PAP-DAMES-SERIE-7_199_jpg.rf.185e10a2654c504d1b0573ea7deacae7.jpg"
MODEL_ARTIFACTS_FILE_NAME = os.path.join(BUCKET, "model/yolov7/model.tar.gz")

!tar zcvf model.tar.gz sm-deploy/*
!aws s3 cp model.tar.gz $MODEL_ARTIFACTS_FILE_NAME

sm-deploy/best.pt
sm-deploy/requirements.txt
upload: ./model.tar.gz to s3://sagemaker-us-west-2-196327195270/model/yolov7/model.tar.gz


In [152]:
%cd ..

/root/archieves


In [153]:
%ls -al

total 84
drwxr-xr-x  6 root root     6144 Dec 21 14:54 ./
drwx------ 14 root nogroup  6144 Dec 21 14:18 ../
drwxr-xr-x  2 root root     6144 Dec 21 02:19 .ipynb_checkpoints/
drwxr-xr-x  4 root root     6144 Nov 23 03:10 Extras/
drwxr-xr-x  5 root root     6144 Dec 14 19:58 UWH-6/
-rw-r--r--  1 root root     3113 Dec 21 14:54 inference.py
-rw-r--r--  1 root root     2801 Dec 21 09:50 inference_2.py
-rw-r--r--  1 root root     2185 Dec 21 09:50 inference_3.py
-rw-r--r--  1 root root     1961 Sep 27 01:13 input_fn.py
drwxr-xr-x 16 root root     6144 Dec 21 14:28 yolov7/
-rw-r--r--  1 root root    21179 Dec 21 14:46 yolov7-Copy1.ipynb
-rw-r--r--  1 root root    20024 Nov 23 03:10 yolov7-Copy2_preprocessing.ipynb


In [158]:
pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-west-2-196327195270/model/yolov7/model.tar.gz', role=role,
                             entry_point='inference.py', py_version='py38', framework_version='1.12.0')
print("PT model loaded")
ENDPOINT_NAME = "yolov7-demo217"

PT model loaded


In [ ]:
# create a serializer for the data
#from sagemaker.serializers import DataSerializer

# image_serializer = DataSerializer(content_type='application/x-image')

predictor = pytorch_model.deploy(instance_type='ml.g4dn.xlarge', initial_instance_count=1, endpoint_name = ENDPOINT_NAME)

-------

In [156]:
predictor.endpoint_name

'yolov7-demo216'

In [157]:
for i in range(10):
    
    client = boto3.client('sagemaker-runtime')

    content_type = 'application/x-image'
    
    sample_img_url = "https://github.com/roboflow/yolov7/blob/main/inference/images/swimmers/7Collines19_100-PAP-SERIES-RENARD-HERLEM_1249_jpg.rf.b46b28cd9d5537429eb95abc74f5f13c.jpg"

    body = requests.get(sample_img_url).content

    response = client.invoke_endpoint(EndpointName=predictor.endpoint_name, Body=body,ContentType=content_type)
    print(response["Body"].read())

ReadTimeoutError: Read timeout on endpoint URL: "https://runtime.sagemaker.us-west-2.amazonaws.com/endpoints/yolov7-demo216/invocations"

In [ ]:
# from torch_ort import ORTInferenceModule
# model = ORTInferenceModule(model)

# pytorch_model = PyTorchModel(model_data='model.tar.gz', role=role,
#                              entry_point='inference.py', py_version='py3')

# predictor = pytorch_model.deploy(instance_type='ml.c5.4xlarge', initial_instance_count=1)

In [43]:
predictor.delete_endpoint(EndpointName)

NameError: name 'EndpointName' is not defined